In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df1 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q1 2018.csv')
df2 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q2 2018.csv')
df3 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q3 2018.csv')
df4 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q4 2018.csv')

In [ ]:
df2018 = df1.copy()
df2018 = df2018.append([df2, df3, df4], sort=False)
del df1
del df2
del df3
del df4

In [ ]:
df2018.head(10)

In [ ]:
df2018['time'] = pd.to_datetime(df2018.trip_start_time)
df2018.index = df2018.time.dt.date
df2018.index.name = 'index'

dfdf2018 = df2018.dropna(subset=['from_station_name','to_station_name'])
df = df2018.copy()

In [ ]:

station = list(set(list(df.from_station_name.values) + list(df.to_station_name.values)))
date = df.index.unique().values

In [ ]:
route = df.copy()
route = route[['from_station_name', 'to_station_name']]

Separate two-way traveller with one-way traveller for each station

In [ ]:
twoway = route.copy()
twoway = twoway[twoway.from_station_name == twoway.to_station_name]
oneway = route.copy()
oneway = oneway[oneway.from_station_name != oneway.to_station_name]

In [ ]:
twoway_map = twoway.groupby('from_station_name').count().sort_values(by='to_station_name', ascending=False)
print('10 Stations with the highest number of two-way traveller')
twoway_map[:10]

Create a table regarding to the number of bikes leaving ('from_station') and entering ('to_station') each day

In [ ]:
outmap = pd.get_dummies(route.from_station_name).groupby('index').sum()
inmap = pd.get_dummies(route.to_station_name).groupby('index').sum()

In [ ]:
outmap.head(5) # number of bikes leaves the station

In [ ]:
inmap.head(5) # number of bikes entering the station

calculate the number of bikes entering the station minus number of bikes leaving the station
* if the result >= 0 then there are enough bike available in the station to be used next morning
* if the result < 0 then we need crew to return some bikes back to the station from other station

In [ ]:
print('number of station with enough bike to use next morning, aka number of bikes entering > number of bikes leaving the station')
((inmap - outmap)>=0).sum(axis=1)

In [ ]:
print('number of station with less bike to use next morning, or need a crew to return bikes back to station before next morning')
((inmap - outmap)<0).sum(axis=1)

Total unique days in 2017 are 329 days. But as we can see below, some stations always lack of bikes for more than 200 days out of 329 days in a year.

May be we should add more bikes in the stations or require user/member to bring their own bikes

In [ ]:
print('Station and the total number of days in 2017 where stations need more bikes to be returned by the crew every night')
((inmap - outmap)<0).sum(axis=0).sort_values(ascending=False)[:20]

In [ ]:
bike_minus = inmap - outmap # incoming bikes minus leaving bikes
bike_minus = np.absolute(bike_minus[bike_minus < 0]) # show only minus value

In [ ]:
bike_minus.head(10) # number of bikes that required by crew to be returned to each station

In [ ]:
print('20 Stations with the highest number of required returned bikes in a day')
np.max(bike_minus, axis=0).sort_values(ascending=False)[:20]